In [3]:
import torch, transformers
import torch.nn as nn 
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoConfig
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

In [4]:
second_test = [['a', 0.1, 'a'], ['b', 4, 'b'], ['c', 1, 'c'], ['']]
first_test = [['a', 0.1, 'a'], ['b', 0, 'b'], ['c', 1, 'c']]

[['a', 0.1, 'a'], ['b', 0, 'b'], ['c', 1, 'c']]

In [7]:
x1 = torch.Tensor([1, 0, 3])
x2 = torch.Tensor([1, 2, 3])

In [2]:
train_df = pd.read_csv('./HappyWhale_YOLO5_Cropped_Dataset/train.csv')
test_df = pd.read_csv('./HappyWhale_YOLO5_Cropped_Dataset/test.csv')

In [6]:
train_df = train_df[['image', 'species', 'individual_id', 'image_path']]
train_df

,image,species,individual_id,image_path
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,../input/happy-whale-and-dolphin/train_images/...
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,../input/happy-whale-and-dolphin/train_images/...
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,../input/happy-whale-and-dolphin/train_images/...
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,../input/happy-whale-and-dolphin/train_images/...
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,../input/happy-whale-and-dolphin/train_images/...
...,...,...,...,...
51028,fff639a7a78b3f.jpg,beluga_whale,5ac053677ed1,../input/happy-whale-and-dolphin/train_images/...
51029,fff8b32daff17e.jpg,cuviers_beaked_whale,1184686361b3,../input/happy-whale-and-dolphin/train_images/...
51030,fff94675cc1aef.jpg,blue_whale,5401612696b9,../input/happy-whale-and-dolphin/train_images/...
51031,fffbc5dd642d8c.jpg,beluga_whale,4000b3d7c24e,../input/happy-whale-and-dolphin/train_images/...


In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51033 entries, 0 to 51032
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   image          51033 non-null  object
 1   species        51033 non-null  object
 2   individual_id  51033 non-null  object
 3   image_path     51033 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


In [12]:
""" Append Image Path """
kaggle_path = '../input/HappyWhale_YOLO5_Cropped_Dataset/train/'

new_path = [kaggle_path + str(train_df.iloc[i, 0]) for i in range(len(train_df))]
train_df['image_path'] = new_path
train_df

/var/folders/l1/l_9fgbw95hl79rj750047d1c0000gn/T/ipykernel_3062/2012963773.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['image_path'] = new_path


,image,species,individual_id,image_path
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...
...,...,...,...,...
51028,fff639a7a78b3f.jpg,beluga_whale,5ac053677ed1,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...
51029,fff8b32daff17e.jpg,cuviers_beaked_whale,1184686361b3,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...
51030,fff94675cc1aef.jpg,blue_whale,5401612696b9,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...
51031,fffbc5dd642d8c.jpg,beluga_whale,4000b3d7c24e,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...


In [18]:
train_df.iloc[1:2, 3].values

array(['../input/HappyWhale_YOLO5_Cropped_Dataset/train/000562241d384d.jpg'],
      dtype=object)

In [19]:
train_df.to_csv('train_df.csv', index=False)

In [24]:
""" Let's Make Test DataFrame """
test_df = test_df[['image', 'image_path']]

In [26]:
test_df['individual_id'] = 0

/var/folders/l1/l_9fgbw95hl79rj750047d1c0000gn/T/ipykernel_3062/3971980077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['individual_id'] = 0


In [27]:
test_df

,image,image_path,individual_id
0,000110707af0ba.jpg,../input/happy-whale-and-dolphin/test_images/0...,0
1,0006287ec424cb.jpg,../input/happy-whale-and-dolphin/test_images/0...,0
2,000809ecb2ccad.jpg,../input/happy-whale-and-dolphin/test_images/0...,0
3,00098d1376dab2.jpg,../input/happy-whale-and-dolphin/test_images/0...,0
4,000b8d89c738bd.jpg,../input/happy-whale-and-dolphin/test_images/0...,0
...,...,...,...
27951,fff6ff1989b5cd.jpg,../input/happy-whale-and-dolphin/test_images/f...,0
27952,fff8fd932b42cb.jpg,../input/happy-whale-and-dolphin/test_images/f...,0
27953,fff96371332c16.jpg,../input/happy-whale-and-dolphin/test_images/f...,0
27954,fffc1c4d3eabc7.jpg,../input/happy-whale-and-dolphin/test_images/f...,0


In [3]:
test_kaggle_path = '../input/HappyWhale_YOLO5_Cropped_Dataset/test/'
test_df = pd.read_csv('./HappyWhale_YOLO5_Cropped_Dataset/test_df.csv')
test_df

,image,image_path,individual_id
0,000110707af0ba.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...,0
1,0006287ec424cb.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...,0
2,000809ecb2ccad.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...,0
3,00098d1376dab2.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...,0
4,000b8d89c738bd.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...,0
...,...,...,...
27951,fff6ff1989b5cd.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...,0
27952,fff8fd932b42cb.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...,0
27953,fff96371332c16.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...,0
27954,fffc1c4d3eabc7.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/trai...,0


In [4]:
new_test_path = [test_kaggle_path + str(test_df.iloc[i, 0]) for i in range(len(test_df))]
new_test_path

['../input/HappyWhale_YOLO5_Cropped_Dataset/test/000110707af0ba.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/0006287ec424cb.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/000809ecb2ccad.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/00098d1376dab2.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/000b8d89c738bd.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/000e246888710c.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/000eb6e73a31a5.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/000fe6ebfc9893.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/0011f7a65044e4.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/0012ff300032e3.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/00150406ce5395.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/0016cd18d6410e.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/0017a9c11c61a8.jpg',
 '../input/HappyWhale_YOLO5_Cropped_Dataset/test/0017f7e0de07b1.jpg',
 '../input/HappyWhal

In [5]:
test_df['image_path'] = new_test_path
test_df

,image,image_path,individual_id
0,000110707af0ba.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/test...,0
1,0006287ec424cb.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/test...,0
2,000809ecb2ccad.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/test...,0
3,00098d1376dab2.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/test...,0
4,000b8d89c738bd.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/test...,0
...,...,...,...
27951,fff6ff1989b5cd.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/test...,0
27952,fff8fd932b42cb.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/test...,0
27953,fff96371332c16.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/test...,0
27954,fffc1c4d3eabc7.jpg,../input/HappyWhale_YOLO5_Cropped_Dataset/test...,0


In [6]:
test_df.to_csv('test_df.csv', index=False)